In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import parmap
from itertools import product
import sys
sys.path.append('../scripts/')
sys.path.append('../examples/')

from lindblad_solver import lindblad_solver
from hamiltonians import single_carbon_H
from utils import sx, sy, sz, si, init_qubit, normal_autocorr_generator, pi_rotation
from dynamical_decoupling import dynamical_decoupling
from matplotlib import pyplot as plt

In [3]:
def generate_parameters(N0):
    wL = 1.0
    wh = 1.06
    theta = np.pi/4

    N, tau0, phi = pi_rotation(wL, wh, theta)
    dt = tau0/N0
    return dt
    
def protect_carbon(H, N0, nf_add, *args, e_ops=[]):
#     e_ops = [np.kron(si, sx), np.kron(si, sy), np.kron(si, sz),np.kron(sx, si), np.kron(sy, si), np.kron(sz, si)]

    wL = 1.0
    wh = 1.06
    theta = np.pi/4
    N, tau0,phi = pi_rotation(wL, wh, theta)
    

    Nf = nf_add + 2*N*N0
    dt = tau0/N0

    rho = np.kron(init_qubit([0,0,1]), init_qubit([1,0,0]))

    tf = (Nf-2*N*N0)*dt
    tlist = np.arange(0,tf,dt)
    
    exp = []
    
    if len(e_ops):
        rho, e = lindblad_solver(H, rho, tlist, *args,e_ops=e_ops)  
        exp.append(e)   
    else:
        rho, e = lindblad_solver(H, rho, tlist, *args,e_ops=[])  

    
    rho = np.kron(sx,si) @ rho @ np.kron(sx,si)
    if len(e_ops):
        rho,_,e = dynamical_decoupling(H, rho, N, tau0, N0, *args,e_ops=e_ops)
        exp.append(e)    
    else:
        rho = dynamical_decoupling(H, rho, N, tau0, N0, *args,e_ops=[])
    rho = np.kron(sx,si) @ rho @ np.kron(sx,si)
    
    
    if len(e_ops):
        rho, e = lindblad_solver(H, rho, tlist, *args,e_ops=e_ops)  
        exp.append(e)   
    else:
        rho, e = lindblad_solver(H, rho, tlist, *args,e_ops=[])  

    if len(e_ops):
        return 2*tf, (np.trace(rho @ np.kron(si,sx))), exp
    else:
        return 2*Nf*dt, (np.trace(rho @ np.kron(si,sx)))

In [7]:
%matplotlib widget
N0 = 5
wL = 1.0
wh = 1.06
theta = np.pi/4


e_ops = [np.kron(si, sx), np.kron(si, sy), np.kron(si, sz)]

t = []
proj_x = []
exp = []
for i in range(5,100,10):
    time, Px, e = protect_carbon(single_carbon_H, N0, i, wL, wh, theta, e_ops = e_ops)
    t.append(time)
    proj_x.append(Px)
    exp.append(e)
plt.plot(t, proj_x)
plt.show()

../scripts/lindblad_solver.py:98: ComplexWarning: Casting complex values to real discards the imaginary part
  expectations[:, i] = np.trace(rho @ e_ops_np, axis1=1, axis2=2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/opt/conda/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [4]:
%matplotlib widget
nf_add = np.arange(5,100,1)
N0 = 5
wL = 1.0
wh = 1.06
theta = np.pi/4
e_ops = [np.kron(si, sx), np.kron(si, sy), np.kron(si, sz),np.kron(sx, si), np.kron(sy, si), np.kron(sz, si)]

parameters = list(
    product([single_carbon_H], [N0], nf_add, [wL], [wh], [theta]))


results = parmap.starmap(protect_carbon,
                          parameters,
                          pm_pbar=True,
                          pm_chunksize=3)

results = np.array(results)
plt.plot(results[:,0], results[:,1])
plt.ylim(0,1.1)
plt.show()

96it [00:00, 172.35it/s]              


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/opt/conda/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [5]:
%matplotlib widget
n = [0]
for result in results[-1,2]:
    print(result.shape)
    n = np.array(range(result.shape[1])) + n[-1]
    plt.plot(n, result[0,:])

IndexError: index 2 is out of bounds for axis 1 with size 2

In [6]:
def single_carbon_H_noise(t, dw_iterator):
    """
    Definition of the Hamiltonian for a single Carbon near a
    Nitrogen-Vacancy centre in diamond.

    Input:
    wL - the Larmor frequency of precession, controlled by the
    externally applied B field

    wh - the hyperfine coupling term describing the strength of
    spin-spin interaction between the Carbon and the NV

    theta - the angle between the applied B field and the vector
    pointing from the NV to the Carbon atom

    Output:
    The 4x4 Hamiltonian of the joint spin system.
    """
    dw = next(dw_iterator)
    wL = 1.0 + dw
    wh = 1.06 
    theta = np.pi/4
    A = wh * np.cos(theta)
    B = wh * np.sin(theta)
    return (A + wL ) * np.kron((si - sz) / 2, sz / 2) + (B) * np.kron(
        (si - sz) / 2, sx / 2) + (wL ) * np.kron((si + sz) / 2, sz / 2) 

In [7]:
def protect_carbon_wrapper(H, nf_add, N0, mu, sigma, corr_time, seed):
    e = []

    # Initial state
#     rho_0 = np.kron(init_qubit([0, 0, 1]), init_qubit([1,0,0]))

    dt = generate_parameters(N0)
    t = []
    e = []
    
    for nf in nf_add:
        dw_it = normal_autocorr_generator(mu, sigma, corr_time / dt / 2, seed)
        time , exp = protect_carbon(H, N0, nf, dw_it)
        t.append(time)
        e.append(exp)
        
    return t, e

In [8]:
nf_add = np.arange(5,100,10)
protect_carbon_wrapper(single_carbon_H_noise, nf_add, 25, 0, 1, 100, 1)

([37.019957576221024,
  37.93403060279438,
  38.84810362936774,
  39.7621766559411,
  40.67624968251446,
  41.590322709087815,
  42.50439573566118,
  43.41846876223453,
  44.332541788807895,
  45.24661481538125],
 [(0.6469773244006066+0j),
  (0.2869255907024334+0j),
  (-0.21494225414633078+0j),
  (-0.6382041270054003+0j),
  (-0.9697895559046624+0j),
  (-0.8641761381573336+0j),
  (-0.4908894834701483+0j),
  (-0.12510923122119114+0j),
  (-0.04867262488386793+0j),
  (-0.3851594127342143+0j)])

In [9]:
def protect_carbon_parallel_noisy(H, nf_add, N0, mu, sigma, corr_time, repetitions):
    seed_list = np.arange(repetitions)

    values = list(product([H], [nf_add], [N0], [mu], [sigma], [corr_time], seed_list))

    results = parmap.starmap(protect_carbon_wrapper, values, pm_chunksize=1, pm_pbar=True)
    results = np.array(results)
    time = results[0,0,:]
    exp_mean = results[:,1,:].mean(axis=0)
    exp_std = results[:,1,:].std(axis=0) / np.sqrt(repetitions - 1)
    print(results.shape)

    return time, exp_mean, exp_std

In [10]:
nf_add = np.arange(5,2000,40)
time, exp_mean, exp_std = protect_carbon_parallel_noisy(single_carbon_H_noise, nf_add, 5, 0, 0.05, 100000, 100)

100%|██████████| 100/100 [04:03<00:00,  2.44s/it]

(100, 2, 50)


In [11]:
%matplotlib widget
plt.errorbar(time, exp_mean, exp_std)
plt.ylim(0,1.1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/opt/conda/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [19]:
%matplotlib widget
data = np.load('../script_output/protecting_carbon_1.npz')
time_1 = data['time']
exp_mean_1 = data['exp_mean']
exp_std_1= data['exp_std']
plt.errorbar(time_1, exp_mean_1, exp_std_1)
plt.ylim(0,1.1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/opt/conda/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [31]:
np.savez('../script_output/protecting_carbon_1',time=time,exp_mean = exp_mean, exp_std = exp_std)

In [12]:
def protect_carbon_2(H, N0, nf_add, sequences, *args, e_ops=[]):
#     e_ops = [np.kron(si, sx), np.kron(si, sy), np.kron(si, sz),np.kron(sx, si), np.kron(sy, si), np.kron(sz, si)]

    wL = 1.0
    wh = 1.06
    theta = np.pi/4
    N, tau0, phi = pi_rotation(wL, wh, theta)
    

    Nf = nf_add + N*N0
    dt = tau0/N0

    rho = np.kron(init_qubit([0,0,1]), init_qubit([1,0,0]))

    tf = nf_add*dt
    tlist = np.arange(0,tf,dt)
    
    exp = []
    # 1st sequence
    # free evol
    for i in range(sequences):
        if len(e_ops):
            rho, e = lindblad_solver(H, rho, tlist, *args,e_ops=e_ops)  
            exp.append(e)   
        else:
            rho, e = lindblad_solver(H, rho, tlist, *args,e_ops=[])  

        # Pi rot
        rho = np.kron(sx,si) @ rho @ np.kron(sx,si)
        if len(e_ops):
            rho,_,e = dynamical_decoupling(H, rho, N, tau0, N0, *args,e_ops=e_ops)
            exp.append(e)    
        else:
            rho = dynamical_decoupling(H, rho, N, tau0, N0, *args,e_ops=[])
        rho = np.kron(sx,si) @ rho @ np.kron(sx,si)

        #Free evol
        if len(e_ops):
            rho, e = lindblad_solver(H, rho, tlist, *args,e_ops=e_ops)  
            exp.append(e)   
        else:
            rho, e = lindblad_solver(H, rho, tlist, *args,e_ops=[])  



  

    
    # Retrun values

    if len(e_ops):
        return 2*sequences*Nf*dt, 1/2 + (np.trace(rho @ np.kron(si,sx)))/2, exp
    else:
        return 2*sequences*Nf*dt, 1/2 + (np.trace(rho @ np.kron(si,sx)))/2
    
def protect_carbon_wrapper_2(H, nf_add,sequences, N0, mu, sigma, corr_time, seed):
    e = []

    # Initial state
#     rho_0 = np.kron(init_qubit([0, 0, 1]), init_qubit([1,0,0]))

    dt = generate_parameters(N0)
    t = []
    e = []
    
    for nf in nf_add:
        dw_it = normal_autocorr_generator(mu, sigma, corr_time / dt / 2, seed)
        time , exp = protect_carbon_2(H, N0, nf,sequences, dw_it)
        t.append(time)
        e.append(exp)
        
    return t, e

def protect_carbon_parallel_noisy_2(H, nf_add,sequences, N0, mu, sigma, corr_time, repetitions):
    seed_list = np.arange(repetitions)

    values = list(product([H], [nf_add],[sequences], [N0], [mu], [sigma], [corr_time], seed_list))

    results = parmap.starmap(protect_carbon_wrapper_2, values, pm_chunksize=1, pm_pbar=True)
    results = np.array(results)
    time = results[0,0,:]
    exp_mean = results[:,1,:].mean(axis=0)
    exp_std = results[:,1,:].std(axis=0) / np.sqrt(repetitions - 1)
    print(results.shape)

    return time, exp_mean, exp_std

In [13]:
#tester box
nf_add = np.arange(5,3000,300)
time_2, exp_mean_2, exp_std_2 = protect_carbon_parallel_noisy_2(single_carbon_H_noise, nf_add, 2, 5, 0, 0.05, 100000, 96)
np.savez('../script_output/protecting_carbon_sequences_2',time=time_2,exp_mean = exp_mean_2, exp_std = exp_std_2)

100%|██████████| 96/96 [02:08<00:00,  1.34s/it]

(96, 2, 10)


In [ ]:
nf_add = np.arange(5,3000,75)
time_2, exp_mean_2, exp_std_2 = protect_carbon_parallel_noisy_2(single_carbon_H_noise, nf_add, 2, 5, 0, 0.05, 100000, 960)
np.savez('../script_output/protecting_carbon_sequences_2',time=time_2,exp_mean = exp_mean_2, exp_std = exp_std_2)

 61%|██████    | 582/960 [46:11<15:14,  2.42s/it]  

In [65]:
%time
nf_add = np.arange(5,1500,40)
time_4, exp_mean_4, exp_std_4 = protect_carbon_parallel_noisy_2(single_carbon_H_noise, nf_add,4, 5, 0, 0.05, 100000, 960)
np.savez('../script_output/protecting_carbon_sequences_4',time=time_4,exp_mean = exp_mean_4, exp_std = exp_std_4)

100%|██████████| 960/960 [27:30<00:00,  1.72s/it]  

(960, 2, 38)


In [67]:
nf_add = np.arange(5,1000,20)
time_8, exp_mean_8, exp_std_8 = protect_carbon_parallel_noisy_2(single_carbon_H_noise, nf_add,8, 5, 0, 0.05, 100000, 960)
np.savez('../script_output/protecting_carbon_sequences_8',time=time_8,exp_mean = exp_mean_8, exp_std = exp_std_8)

100%|██████████| 960/960 [48:50<00:00,  3.05s/it]  

(960, 2, 50)


In [ ]:
nf_add = np.arange(5,500,10)
time_16, exp_mean_16, exp_std_16 = protect_carbon_parallel_noisy_2(single_carbon_H_noise, nf_add,16, 5, 0, 0.05, 100000, 960)
np.savez('../script_output/protecting_carbon_sequences_16',time=time_16,exp_mean = exp_mean_16, exp_std = exp_std_16)

In [71]:
%matplotlib widget
plt.errorbar(time_1, exp_mean_1, exp_std_1)
plt.errorbar(time_2, exp_mean_2, exp_std_2)
plt.errorbar(time_4, exp_mean_4, exp_std_4)

plt.xscale('log')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/opt/conda/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [61]:
nf_add = np.arange(5,1000,10)
time_4, exp_mean_4, exp_std_4 = protect_carbon_parallel_noisy_2(single_carbon_H_noise, nf_add,4, 5, 0, 0.05, 100000, 96)


100%|██████████| 96/96 [10:02<00:00,  6.28s/it]  

(96, 2, 100)


In [53]:
time_4

array([18.85275617+0.j, 18.9670153 +0.j, 19.08127443+0.j, 19.19553356+0.j,
       19.30979269+0.j])